Práctica: Sistema de medición y control de distancia con visualización local y en la nube

Descripción general

En esta práctica, los estudiantes implementarán un sistema embebido inteligente basado en Raspberry Pi y el sensor ultrasónico HC-SR04, que mide distancias y activa una secuencia de luces LED según la proximidad del objeto.
Además, el sistema incorpora una interfaz web local (desarrollada con Flask) que permite visualizar en tiempo real los valores medidos, el estado de los LEDs y una tabla con los registros recientes.

De forma simultánea, los datos también se envían a la plataforma ThingSpeak, donde se almacenan y grafican para su monitoreo en la nube, integrando así los conceptos de IoT (Internet de las Cosas) y control local embebido.

Secuencia de pasos:

1. Montaje físico del sistema

- Conecta el sensor HC-SR04 a los pines GPIO de la Raspberry Pi (TRIG=23, ECHO=24 con divisor de voltaje 5V→3.3V).

- Conecta los cuatro LEDs (verde, amarillo, naranja y rojo) a los pines GPIO17, GPIO27, GPIO22 y GPIO5 respectivamente, cada uno con su resistencia a GND.

- Verifica las conexiones de alimentación y tierra comunes.


2. Configuración del entorno

- Abre Thonny en la Raspberry Pi.

- Crea una carpeta de proyecto (por ejemplo, Practica_Distancia_Flask_ThingSpeak).

- Guarda el archivo Python (app.py) proporcionado.

- Asegúrate de tener instalado Flask y requests:
sudo apt install python3-flask python3-requests -y


3. Configuración de ThingSpeak

- Ingresa a https://thingspeak.com
 y crea una cuenta gratuita.

- Crea un nuevo canal con al menos un campo (“Field 1”).

- Copia tu Write API Key y reemplázala en el código (THINGSPEAK_API_KEY = "TU_CLAVE").

- (Opcional) Si el canal es público, copia el CHANNEL_ID para incrustar la gráfica en la web local.


4. Ejecución del sistema

- Ejecuta el programa en Thonny (Run ▶ o F5).

- En el navegador de la Raspberry Pi (o desde otro equipo en la misma red), abre:
http://<IP_DE_TU_PI>:5000

- Verás un panel web con:

  - La distancia medida en tiempo real.

  - Indicadores visuales de los LEDs.

  - Una tabla con los últimos 10 registros.

  - Botón para activar/desactivar el envío a ThingSpeak.

  - Gráfica en vivo del canal.


5. Verificación del funcionamiento

- Acerca o aleja un objeto frente al sensor y observa:

  - Cambios en la secuencia de LEDs (verde → verde+amarillo → verde+amarillo+naranja → rojo parpadeante).

  - Actualización inmediata de la lectura en la web.

  - Registro automático en la tabla y archivo CSV.

  - Envío periódico de los datos a ThingSpeak (cada 15 s).

In [ ]:
# HC-SR04 + LEDs + Flask + ThingSpeak + CSV + Tabla (últimos 10)
from flask import Flask, render_template_string, jsonify, request
import RPi.GPIO as GPIO, time, threading, requests, os, csv
from statistics import mean
from collections import deque
from datetime import datetime

# ---------- Pines (BCM) ----------
GPIO.setmode(GPIO.BCM)
TRIG, ECHO = 23, 24
LED_V, LED_A, LED_N, LED_R = 17, 27, 22, 5
for p in [TRIG, LED_V, LED_A, LED_N, LED_R]:
    GPIO.setup(p, GPIO.OUT)
GPIO.setup(ECHO, GPIO.IN)
for p in [LED_V, LED_A, LED_N, LED_R]:
    GPIO.output(p, 0)

# ---------- Umbrales (cm) ----------
T1, T2, T3, T4 = 40, 25, 15, 8  # ajustables

# ---------- ThingSpeak ----------
THINGSPEAK_API_KEY = "TU_WRITE_API_KEY"
CHANNEL_ID = "TU_CHANNEL_ID"
UPLOAD_PERIOD = 15  # segundos
cloud_enabled = True  # alternable desde la web

# ---------- Registro CSV ----------
DATA_DIR = "data"
CSV_PATH = os.path.join(DATA_DIR, "log.csv")
os.makedirs(DATA_DIR, exist_ok=True)
if not os.path.exists(CSV_PATH):
    with open(CSV_PATH, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["timestamp", "distance_cm", "led_V", "led_A", "led_N", "led_R"])

# ---------- Estado compartido ----------
state = {
    "dist": None,
    "leds": {"V":0, "A":0, "N":0, "R":0},
    "last_upload": None,
    "cloud_enabled": cloud_enabled
}
recent = deque(maxlen=10)  # últimos 10 registros para la tabla
lock = threading.Lock()

def set_leds(v=0,a=0,n=0,r=0):
    GPIO.output(LED_V, v); GPIO.output(LED_A, a)
    GPIO.output(LED_N, n); GPIO.output(LED_R, r)
    with lock:
        state["leds"] = {"V":int(v),"A":int(a),"N":int(n),"R":int(r)}

def medir_distancia_cm(timeout=0.02):
    GPIO.output(TRIG, 0); time.sleep(0.0002)
    GPIO.output(TRIG, 1); time.sleep(0.00001)
    GPIO.output(TRIG, 0)
    t0 = time.time()
    while GPIO.input(ECHO) == 0:
        if time.time()-t0 > timeout: return None
    start = time.time()
    while GPIO.input(ECHO) == 1:
        if time.time()-start > timeout: return None
    end = time.time()
    return (end-start)*34300/2

def distancia_suavizada(N=5, pausa=0.03):
    vals, tries = [], 0
    while len(vals)<N and tries<N*3:
        d = medir_distancia_cm()
        if d and 2<d<400: vals.append(d)
        tries += 1; time.sleep(pausa)
    return mean(vals) if vals else None

def registrar_csv(timestamp, dist, leds):
    with open(CSV_PATH, "a", newline="") as f:
        w = csv.writer(f)
        w.writerow([timestamp, f"{dist:.2f}", leds["V"], leds["A"], leds["N"], leds["R"]])

def control_loop():
    blink = False; tblink = time.time(); t_log = 0
    while True:
        d = distancia_suavizada()
        now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        if d is None:
            set_leds(0,0,0,0)
        else:
            # Secuencia por umbrales
            if d > T1:
                set_leds(0,0,0,0)
            elif T2 < d <= T1:
                set_leds(1,0,0,0)
            elif T3 < d <= T2:
                set_leds(1,1,0,0)
            elif T4 < d <= T3:
                set_leds(1,1,1,0)
            else:
                # rojo parpadeante no bloqueante
                if time.time() - tblink > 0.1:
                    blink = not blink; tblink = time.time()
                set_leds(0,0,0,1 if blink else 0)

        with lock:
            state["dist"] = d
            # Guardar reciente y CSV cada ~0.5 s (si hay dato)
            if d is not None and (time.time() - t_log) > 0.5:
                rec = {
                    "timestamp": now,
                    "distance": float(f"{d:.2f}"),
                    "leds": state["leds"].copy()
                }
                recent.appendleft(rec)  # más nuevo primero
                registrar_csv(now, d, state["leds"])
                t_log = time.time()

        time.sleep(0.02)

def cloud_loop():
    while True:
        time.sleep(UPLOAD_PERIOD)
        with lock:
            if not state["cloud_enabled"] or state["dist"] is None:
                continue
            dist = round(state["dist"], 2)
        try:
            url = f"https://api.thingspeak.com/update?api_key={THINGSPEAK_API_KEY}&field1={dist}"
            r = requests.get(url, timeout=5)
            if r.status_code == 200 and r.text.strip().isdigit():
                with lock: state["last_upload"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        except Exception:
            pass

# ---------- Flask ----------
HTML = """
<!doctype html>
<meta charset="utf-8">
<title>Proximidad: Panel Local + Nube</title>
<style>
  body{font-family:Arial;max-width:1100px;margin:24px auto;}
  .cards{display:flex;gap:16px;flex-wrap:wrap}
  .card{padding:16px;border:1px solid #ddd;border-radius:12px;flex:1;min-width:280px}
  .leds span{display:inline-block;width:18px;height:18px;border-radius:50%;margin-right:6px;vertical-align:middle}
  .on{box-shadow:0 0 8px rgba(0,0,0,0.25)}
  #V{background:#00b050} #A{background:#ffd966} #N{background:#f6b26b} #R{background:#e06666}
  button{padding:8px 14px;border-radius:10px;border:1px solid #bbb;background:#f4f4f4;cursor:pointer}
  table{width:100%;border-collapse:collapse;margin-top:10px}
  th,td{border:1px solid #ddd;padding:8px;text-align:center}
  th{background:#f7f7f7}
  iframe{width:100%;height:300px;border:0}
</style>
<h2>🔹 Panel de Proximidad (HC-SR04 + LEDs + ThingSpeak + CSV)</h2>
<div class="cards">
  <div class="card">
    <h3>Distancia</h3>
    <div id="dist" style="font-size:24px">-- cm</div>
    <small>Último envío a ThingSpeak: <span id="last">--</span></small>
  </div>
  <div class="card">
    <h3>Secuencia LEDs</h3>
    <div class="leds">
      <span id="V" class=""></span> Verde
      <span id="A" class=""></span> Amarillo
      <span id="N" class=""></span> Naranja
      <span id="R" class=""></span> Rojo
    </div>
  </div>
  <div class="card">
    <h3>Nube</h3>
    <div>ThingSpeak: <b id="cloud">--</b></div>
    <button onclick="toggleCloud()">Alternar envío</button>
  </div>
</div>

<h3>📄 Últimos 10 registros (CSV local)</h3>
<table>
  <thead>
    <tr><th>Fecha/Hora</th><th>Distancia (cm)</th><th>V</th><th>A</th><th>N</th><th>R</th></tr>
  </thead>
  <tbody id="rows"></tbody>
</table>

<h3>📈 Gráfica ThingSpeak (canal público)</h3>
<p>Reemplaza CHANNEL_ID por el tuyo si quieres ver la gráfica embebida.</p>
<iframe src="https://thingspeak.com/channels/CHANNEL_ID/charts/1?bgcolor=%23ffffff&dynamic=true&results=60&title=Distancia+cm&type=line"></iframe>

<script>
async function poll(){
  const r = await fetch('/status'); const j = await r.json();
  document.getElementById('dist').innerText = j.dist? (j.dist.toFixed(1)+' cm') : '-- cm';
  document.getElementById('last').innerText = j.last_upload || '--';
  document.getElementById('cloud').innerText = j.cloud_enabled? 'ON' : 'OFF';
  // LEDs
  for (const k of ['V','A','N','R']){
    const el = document.getElementById(k);
    if (j.leds[k]) el.classList.add('on'); else el.classList.remove('on');
  }
  // Tabla
  const tbody = document.getElementById('rows'); tbody.innerHTML = '';
  (j.recent || []).forEach(row=>{
    const tr = document.createElement('tr');
    tr.innerHTML = `<td>${row.timestamp}</td>
                    <td>${row.distance.toFixed(2)}</td>
                    <td>${row.leds.V}</td>
                    <td>${row.leds.A}</td>
                    <td>${row.leds.N}</td>
                    <td>${row.leds.R}</td>`;
    tbody.appendChild(tr);
  });
}
async function toggleCloud(){ await fetch('/cloud_toggle', {method:'POST'}); }
setInterval(poll, 500); poll();
</script>
"""

app = Flask(__name__)

@app.route("/")
def index():
    return render_template_string(HTML)

@app.route("/status")
def status():
    with lock:
        return jsonify({
            "dist": state["dist"],
            "leds": state["leds"],
            "last_upload": state["last_upload"],
            "cloud_enabled": state["cloud_enabled"],
            "recent": list(recent)  # ya están en orden (nuevo primero)
        })

@app.route("/cloud_toggle", methods=["POST"])
def cloud_toggle():
    with lock:
        state["cloud_enabled"] = not state["cloud_enabled"]
    return ("", 204)

# ---------- Run ----------
if __name__ == "__main__":
    try:
        threading.Thread(target=control_loop, daemon=True).start()
        threading.Thread(target=cloud_loop, daemon=True).start()
        print("Abre el navegador en: http://0.0.0.0:5000 (o http://<IP_DE_TU_PI>:5000)")
        app.run(host="0.0.0.0", port=5000)
    finally:
        set_leds(0,0,0,0); GPIO.cleanup()
